# Importando as Bibliotecas

In [73]:
"""Phraser e Phrases são utilizados para extrair as sequências de palavras
   mais frequentes na coleção"""
from gensim.models.phrases import Phraser, Phrases

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import spacy

In [33]:
pd.set_option('max_colwidth',200)
nlp = spacy.load("en_core_web_sm")

In [53]:
def tokenizador(text): 
  doc = nlp(text)
  return [token.lemma_.lower() for token in doc if (not token.is_stop) and (token.is_alpha)]

In [54]:
""" Função utilizada para criar uma lista de listas de tokens.
    O retorno será utilizado como entrada para o Phrases"""
def tokenizador_para_lista(textos): 
  lista = []
  for texto in textos:
    lista_temp = tokenizador(texto)
    lista.append(lista_temp)
  return lista

In [55]:
#Criando a base de dados
texts = ['Goku is a hero in the Dragon Ball since 1989! Goku saved the earth so many times.',
         'The 7 Dragon balls can make wishes come true! Each ball contains his own dragon.',
         'If the wishes are superfluous, the dragon balls will become dark.' ,
         'Seiya is a bronze knight and is one of the main Knights of the Zodiac. He saved Athena several times.',
         "A knight of the zodiac wear a bronze, silver or a gold cloth to protect Athena.",
         'Saint Seiya: Knights of the Zodiac is a Japanese manga in which mystical warriors called the Saints fight wearing sacred cloths.']
classes = ['Dragon Ball', 'Dragon Ball', 'Dragon Ball', 'Cav. Zod.', 'Cav. Zod.', 'Cav. Zod.']

df = pd.DataFrame({'texts': texts, 'classes': classes})

In [58]:
#Exibindo o resultado só da função tokenizadora
tokenizador(df['texts'].loc[0])

['goku', 'hero', 'dragon', 'ball', 'goku', 'save', 'earth', 'time']

In [59]:
#Exibindo o resultado da função utilizada para gerar uma lista de listas de tokens
tokenizador_para_lista(df['texts'])

[['goku', 'hero', 'dragon', 'ball', 'goku', 'save', 'earth', 'time'],
 ['dragon', 'ball', 'wish', 'come', 'true', 'ball', 'contain', 'dragon'],
 ['wish', 'superfluous', 'dragon', 'ball', 'dark'],
 ['seiya',
  'bronze',
  'knight',
  'main',
  'knights',
  'zodiac',
  'save',
  'athena',
  'time'],
 ['knight',
  'zodiac',
  'wear',
  'bronze',
  'silver',
  'gold',
  'cloth',
  'protect',
  'athena'],
 ['saint',
  'seiya',
  'knights',
  'zodiac',
  'japanese',
  'manga',
  'mystical',
  'warrior',
  'call',
  'saint',
  'fight',
  'wear',
  'sacred',
  'cloth']]

In [60]:
"""Crinado um modelo passando como parâmeteros as listas de tokens para cada texto,
   qual a frequencia mínima que um token deve aparecer para ser considera no processo,
   e o limiar para ser considerado frequente"""
phrases = Phrases(tokenizador_para_lista(df['texts']), min_count=1, threshold=1)

In [61]:
"""Criando um objeto contendo as frases mais frequentes
   A partir dele é possível substituir os tokens individuais por sequências de tokens
   frequentes."""
phraser = Phraser(phrases)

In [62]:
novo_texto = 'I watched Dragon Ball every morning'
tokenizador(novo_texto)

['watch', 'dragon', 'ball', 'morning']

In [63]:
phraser[tokenizador(novo_texto)]

['watch', 'dragon_ball', 'morning']

In [68]:
def preprocessador(texto): 
  return ' '.join(phraser[tokenizador(texto)])

In [69]:
df['texts_preprocessed'] = df['texts'].apply(preprocessador)

In [70]:
df

,texts,classes,texts_preprocessed
0,Goku is a hero in the Dragon Ball since 1989! Goku saved the earth so many times.,Dragon Ball,goku hero dragon_ball goku save earth time
1,The 7 Dragon balls can make wishes come true! Each ball contains his own dragon.,Dragon Ball,dragon_ball wish come true ball contain dragon
2,"If the wishes are superfluous, the dragon balls will become dark.",Dragon Ball,wish superfluous dragon_ball dark
3,Seiya is a bronze knight and is one of the main Knights of the Zodiac. He saved Athena several times.,Cav. Zod.,seiya bronze knight main knights_zodiac save athena time
4,"A knight of the zodiac wear a bronze, silver or a gold cloth to protect Athena.",Cav. Zod.,knight zodiac wear bronze silver gold cloth protect athena
5,Saint Seiya: Knights of the Zodiac is a Japanese manga in which mystical warriors called the Saints fight wearing sacred cloths.,Cav. Zod.,saint seiya knights_zodiac japanese manga mystical warrior call saint fight wear sacred cloth


In [74]:
#Criando o vetorizador e já obtendo o vocabulário e gerando a representação estruturada
vetorizador = CountVectorizer(min_df=2)
representacao = vetorizador.fit_transform(df['texts_preprocessed'])

In [75]:
#Obtendo os termos ordenados de acordo com o seu índice no vocabulário
colunas = [item[0] for item in sorted(vetorizador.vocabulary_.items(), key = lambda x: x[1])]

#Gerando o DataFrame com o dados
df_repr = pd.DataFrame(representacao.toarray(), columns=colunas)

In [76]:
df_repr

,athena,bronze,cloth,dragon_ball,knight,knights_zodiac,save,seiya,time,wear,wish
0,0,0,0,1,0,0,1,0,1,0,0
1,0,0,0,1,0,0,0,0,0,0,1
2,0,0,0,1,0,0,0,0,0,0,1
3,1,1,0,0,1,1,1,1,1,0,0
4,1,1,1,0,1,0,0,0,0,1,0
5,0,0,1,0,0,1,0,1,0,1,0


In [77]:
df_repr.shape

(6, 11)